In [1]:
from main_model.src.data.generate_problem_data import *

In [2]:
mesh_path = "main_model/disk/meshes/sphere.obj"
problem_size = 3
num_customers = 20

dmd_lower=1
dmd_upper=9
cap_lower=50
cap_upper=50
max_runtime=5

In [3]:
mesh_city, city_size = get_mesh_city(mesh_path, num_customers)

2025-03-25 17:15:16,010 - INFO - Loading mesh from main_model/disk/meshes/sphere.obj
2025-03-25 17:15:16,068 - INFO - Selected depot at vertex index 4296
2025-03-25 17:15:16,069 - INFO - Sampled 20 customer locations from mesh vertices
2025-03-25 17:15:16,069 - INFO - Computing geodesic distance matrix...
Computing geodesic distances: 100%|██████████| 21/21 [00:01<00:00, 14.76it/s]
2025-03-25 17:15:27,404 - INFO - Time taken to compute geodesic distances: 0.19m
2025-03-25 17:15:27,405 - INFO - Finished computing geodesic distance matrix


In [4]:
# load and print the object defined by the vertices and faces
mesh = trimesh.Trimesh(vertices=mesh_city["vertices"], faces=mesh_city["faces"])
mesh.show()

In [5]:
# check if each mapped city indices has the correct coordinates
for i in range(city_size):
    print(mesh_city["vertices"][mesh_city["city_indices"][i]] == mesh_city["city"][i])

[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]


In [6]:
import numpy as np

def true_distance(p1, p2, R=1):
    """
    Compute the great-circle distance between two points on a sphere.

    Parameters:
        p1, p2 : array-like
            Cartesian coordinates of the two points (x, y, z), assumed to be on a sphere of radius R.
        R : float
            Radius of the sphere (default is 1 for a unit sphere).

    Returns:
        float : Geodesic distance along the sphere.
    """
    dot_product = np.dot(p1, p2)
    theta = np.arccos(np.clip(dot_product, -1.0, 1.0))  # Clip to avoid numerical issues
    return R * theta

def geodesic_path(p1, p2, num_points=100):
    """
    Compute the geodesic path (great-circle arc) between two points on a sphere.

    Parameters:
        p1, p2 : array-like
            Cartesian coordinates of the two points (x, y, z).
        num_points : int
            Number of points to generate along the geodesic.

    Returns:
        np.ndarray : Array of shape (num_points, 3) containing the geodesic path.
    """
    p1, p2 = np.array(p1), np.array(p2)
    dot_product = np.dot(p1, p2)
    theta = np.arccos(np.clip(dot_product, -1.0, 1.0))

    # Generate intermediate points using slerp
    t_values = np.linspace(0, 1, num_points)
    path = np.array([
        (np.sin((1-t)*theta) / np.sin(theta)) * p1 + (np.sin(t*theta) / np.sin(theta)) * p2
        for t in t_values
    ])

    return path

# Compute geodesic path
# path = geodesic_path(p1, p2)

# # Plot the geodesic path
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# fig = plt.figure(figsize=(6,6))
# ax = fig.add_subplot(111, projection='3d')

# # Plot the sphere
# u = np.linspace(0, 2 * np.pi, 100)
# v = np.linspace(0, np.pi, 50)
# x = np.outer(np.cos(u), np.sin(v))
# y = np.outer(np.sin(u), np.sin(v))
# z = np.outer(np.ones_like(u), np.cos(v))
# ax.plot_surface(x, y, z, color='c', alpha=0.3)

# # Plot geodesic path
# ax.plot(path[:, 0], path[:, 1], path[:, 2], 'r-', linewidth=2, label="Geodesic Path")
# ax.scatter(*p1, color='b', s=100, label="Point 1")
# ax.scatter(*p2, color='g', s=100, label="Point 2")

# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.legend()
# plt.show()


In [7]:
depot = mesh_city["city"][0]
geodesic_mat = mesh_city["geodesic_matrix"]
for i, customer in enumerate(mesh_city["city"]):
    print(true_distance(depot, customer) - geodesic_mat[0, i])

0.0003452669847162036
0.0003129577015745788
0.0003762152555069065
0.00025274762508886717
0.00024117992673056143
4.76755841120613e-05
0.0003175545629297005
0.00016066237410994333
0.00034691168109057813
0.0002711660603094135
0.00032097282163157104
0.00014167985918733805
2.5888056640832602e-05
0.00030439453254405
0.00022474308417952393
7.242600896945062e-05
0.00017533225392685203
0.000104656343191456
4.041029130696927e-05
0.0002536303409079732
0.0003273084069088661


In [8]:
problem = get_problem(
        city_size, problem_size, dmd_lower, dmd_upper, cap_lower, cap_upper
    )

In [9]:
problem

{'problem': array([ 0, 17, 20, 11], dtype=int32),
 'demand': array([0, 7, 8, 7], dtype=int32),
 'capacity': 50}

In [10]:
solution = get_solution(mesh_city, problem, max_runtime)

In [56]:
problem_indices = problem["problem"]

# Get the precomputed geodesic matrix
geodesic_matrix = mesh_city["geodesic_matrix"]

# Set up PyVRP model with custom distance function
m = Model()
m.add_vehicle_type(1000, capacity=problem["capacity"])

demands = problem["demand"].tolist()

import random
random.randint(1, 10)

depot = m.add_depot(x=0, y=0)
clients = [
    m.add_client(x=random.randint(1, 10), y=random.randint(1, 10), delivery=demands[idx])
    for idx in range(1, len(problem_indices))
]

# Add edges with precomputed geodesic distances
locations = [depot] + clients
for i, frm in enumerate(locations):
    for j, to in enumerate(locations):
        if i != j:
            distance = geodesic_matrix[problem_indices[i], problem_indices[j]]
            m.add_edge(frm, to, distance=int(distance * 1000))

# Solve and return solution
res = m.solve(stop=MaxRuntime(max_runtime), display=False)

In [57]:
routes = res.best.routes()
solution = [0]
for route in routes:
    solution += route.visits()
    solution.append(0)

In [55]:
solution

[0]

In [18]:
# calculate the total route distance by computing true distance between consecutive routes
solution_path = solution["solution"]
problem_indices = problem["problem"]
total_distance = 0
for i in range(len(solution) - 1):
    total_distance += true_distance(mesh_city["city"][problem_indices[solution_path[i]]], mesh_city["city"][problem_indices[solution_path[i + 1]]])

In [19]:
total_distance

3.30439695628208

In [14]:
problem_indices = problem["problem"]

# Get the precomputed geodesic matrix
geodesic_matrix = mesh_city["geodesic_matrix"]

# Map problem indices to city indices for the distance matrix lookup
problem_to_city_map = {p_idx: c_idx for c_idx, p_idx in enumerate(problem_indices)}

# Function to get geodesic distance between two points using the precomputed matrix
def get_distance(i, j):
    city_i = problem_to_city_map[i]
    city_j = problem_to_city_map[j]
    return geodesic_matrix[city_i, city_j]


In [23]:
problem_to_city_map

{0: 0, 5: 1, 9: 2, 10: 3, 7: 4, 6: 5}

In [44]:
# Set up PyVRP model with custom distance function
m = Model()
m.add_vehicle_type(1000, capacity=problem["capacity"])

# Add locations
COORDS = mesh_city["city"][problem_indices]
COORDS = (COORDS[:, :2] * 1000).round().astype(np.int32).tolist()
DEMANDS = problem["demand"].tolist()

depot = m.add_depot(x=0, y=0)
clients = [
    m.add_client(x=0, y=0, delivery=DEMANDS[idx])
    for idx in range(1, len(COORDS))
]


In [45]:

# Add edges with precomputed geodesic distances
locations = [depot] + clients
for i, frm in enumerate(locations):
    for j, to in enumerate(locations):
        if i != j:
            distance = geodesic_matrix[problem_indices[i], problem_indices[j]]
            m.add_edge(frm, to, distance=int(distance * 1000))

In [46]:
# Solve and return solution
res = m.solve(stop=MaxRuntime(max_runtime), display=False)
res_dict = parse_pyvrp_solution(res.best)

# Convert distance back to float
res_dict["distance"] = res_dict["distance"] / 1000

# Convert solution back into the city space
# solution = [problem["problem"][idx] for idx in res_dict["solution"]]
solution = res_dict["solution"]

In [47]:
res_dict["node_flag"] = transform_solution(solution)

In [48]:
res_dict

{'distance': 6.874,
 'is_feasible': True,
 'num_routes': 1,
 'solution': [0, 4, 5, 3, 2, 1, 0],
 'node_flag': [4, 5, 3, 2, 1, 1, 0, 0, 0, 0]}

In [49]:
# calculate the total route distance by computing true distance between consecutive routes
solution_path = res_dict["solution"]
total_distance = 0
for i in range(len(solution_path) - 1):
    total_distance += true_distance(mesh_city["city"][problem_indices[solution_path[i]]], mesh_city["city"][problem_indices[solution_path[i+1]]])

In [50]:
total_distance

6.877830815060449